In [29]:
from pathlib import Path
from itertools import combinations
from fastquadtree import RectQuadTree
import numpy as np
import sys
import time

input_file = Path(".") / "input.txt"

def print_matrix(matrix: list, title: str="Matrix"):
    if title:
        print(title.title())
    print(np.array(matrix), "\n")

def calculate_area(x1, y1, x2, y2: int) -> int:
    a = abs(x1-x2) + 1
    b = abs(y1-y2) + 1
    return a * b

def calcualte_rectangle_areas_sorted(points: list) -> list:
    '''Calculate rectangle areas
    Given a list of points (2D coordinates), calculate the area of rectangles created from point pairs
    were opposite corners of a rectangle are two point from the list.

    Args:
        points (list): list of 2D coordinates represented by a list of 2 integers
        
    Returns:
        list: list of tuples (area, x1, y1, x2, y2) sorted by area in descending order
    '''
    area_of_rectangles = []
    for point1, point2 in list(combinations(points, 2)):
        x1, y1 = point1
        x2, y2 = point2
        area = calculate_area(x1, y1, x2, y2)
        area_of_rectangles.append((area, x1, y1, x2, y2))
    return sorted(area_of_rectangles, reverse=True)

def find_greatest_area_avoiding_exclusions(rectangle_areas: list, tile_map: QuadTree) -> int:
    '''
    Args:
        rectangle_areas (list): list of tuples (area, x1, y1, x2, y2)
        exclusions (QuadTree): key is a 2D point represented by tuple of 2 integers, the value is a boolean, where True means that point is excluded to be covered
    
    Returns:
        int: the greatest area of a rectangle which includes points allowed by the exclusion_map argument
    '''    
    for area, x1, y1, x2, y2 in rectangle_areas:
        #print(f"checking rectagle area {area} for retangle {(x1, y1)} {(x2, y2)}")
        allowed = True
        #s = time.time()
        for x, y in enumerate_rectange_border_coordinates(x1, y1, x2, y2):
            if not tile_map.query((x, y, x, y)):
                allowed = False
                break
        #print(f"took {time.time() - s} sec")
        if allowed:
            return area

    return 0

def find_greatest_area_avoiding_exclusions2(rectangle_areas: list, tile_map: QuadTree) -> int:
    '''
    Args:
        rectangle_areas (list): list of tuples (area, x1, y1, x2, y2)
        exclusions (QuadTree): key is a 2D point represented by tuple of 2 integers, the value is a boolean, where True means that point is excluded to be covered
    
    Returns:
        int: the greatest area of a rectangle which includes points allowed by the exclusion_map argument
    '''
    s = None
    for area, x1, y1, x2, y2 in rectangle_areas:
        #print(f"checking rectagle area {area} for retangle {(x1, y1)} {(x2, y2)}")
        #if s != None:
        #    print(f"took {time.time() - s} sec")
        #s = time.time()

        allowed = False

        top_border = (x1, min(y1, y2), x1, max(y1, y2))
        matches = tile_map.query(top_border)
        for y in range(min(y1, y2), max(y1, y2) + 1):
            allowed = False
            for _, x3, y3, x4, y4 in matches:
                if x3 <= x1 <= x4 and y3 <= y <= y4:
                    allowed = True
                    break
            if not allowed:
                break
        
        if not allowed:
            continue

        bottom_border = (x2, min(y1, y2), x2, max(y1, y2))
        matches = tile_map.query(bottom_border)
        for y in range(min(y1, y2), max(y1, y2) + 1):
            allowed = False
            for _, x3, y3, x4, y4 in matches:
                if x3 <= x2 <= x4 and y3 <= y <= y4:
                    allowed = True
                    break
            if not allowed:
                break

        if not allowed:
            continue

        left_border = (min(x1, x2), y1, max(x1, x2), y1)
        matches = tile_map.query(left_border)
        for x in range(min(x1, x2), max(x1, x2) + 1):
            allowed = False
            for _, x3, y3, x4, y4 in matches:
                if x3 <= x <= x4 and y3 <= y1 <= y4:
                    allowed = True
                    break
            if not allowed:
                break

        if not allowed:
            continue

        right_border = (max(x1, x2), y2, max(x1, x2), y2)
        matches = tile_map.query(right_border)
        for x in range(min(x1, x2), max(x1, x2) + 1):
            allowed = False
            for _, x3, y3, x4, y4 in matches:
                if x3 <= x <= x4 and y3 <= y2 <= y4:
                    allowed = True
                    break
            if not allowed:
                break
        
        if allowed:
            return area

    return 0


def generate_inclusions(red_points: list) -> QuadTree:
    '''Build a 2D map of allowed and excluded points
    Args:
        points (list): list of 2D coordinates represented by a list of 2 integers
    
    Returns:
        QuadTree: points which can be included inside a rectangle
    '''
    RED_TILE = 1
    GREEN_TILE = 2
    LIGHT_GREEN_TILE = 3
    
    # find out the dimensions of the map
    max_x, max_y = 0, 0
    min_x, min_y = sys.maxsize, sys.maxsize
    for x, y in red_points:
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
        if x < min_x:
            min_x = x
        if y < min_y:
            min_y = y
    tile_map = RectQuadTree(bounds=(min_x, min_y, max_x, max_y), capacity=20, dtype="i32")
   
    # mark lines between red tiles (including red) as green
    green_points = []
    green_points += fill_horizontal_line(tile_map, GREEN_TILE, red_points, return_filled_points=True)
    green_points += fill_vertical_line(tile_map, GREEN_TILE, red_points, return_filled_points=True)
    
    # mark lines between green tiles as light green
    fill_horizontal_line(tile_map, LIGHT_GREEN_TILE, green_points)
    fill_vertical_line(tile_map, LIGHT_GREEN_TILE, green_points)

    return tile_map

def fill_vertical_line(tile_map: RectQuadTree, fill_color: int, points: list, return_filled_points: bool = False) -> list:
    X_INDEX = 0
    Y_INDEX = 1
    filled_points = []
    sorted_points = sorted(points, key=lambda point: point[Y_INDEX])
    previous = sorted_points[0]
    for point in sorted_points[1:]:
        if previous[Y_INDEX] == point[Y_INDEX]:
            start = min(previous[X_INDEX], point[X_INDEX])
            end = max(previous[X_INDEX], point[X_INDEX])
            tile_map.insert((start, previous[Y_INDEX], end, point[Y_INDEX]))
            if return_filled_points:                
                for x in range(start + 1, end):
                    filled_points.append([x, point[Y_INDEX]])
        previous = point

    return filled_points

def fill_horizontal_line(tile_map: RectQuadTree, fill_color: int, points: list, return_filled_points: bool = False) -> list:
    X_INDEX = 0
    Y_INDEX = 1
    filled_points = []
    sorted_points = sorted(points, key=lambda point: point[X_INDEX])
    
    previous = sorted_points[0]
    for point in sorted_points[1:]:
        if previous[X_INDEX] == point[X_INDEX]:
            start = min(previous[Y_INDEX], point[Y_INDEX])
            end = max(previous[Y_INDEX], point[Y_INDEX])
            tile_map.insert((previous[X_INDEX], start, point[X_INDEX], end))
            if return_filled_points:
                for y in range(start + 1, end):
                    filled_points.append([point[X_INDEX], y])
        previous = point

    return filled_points

def enumerate_rectange_border_coordinates(x1, y1, x2, y2: int) -> tuple:
    if x1 == x2 and y1 == y2:
        raise ValueError(f"This is not a rectange: {(x1, y1, x2, y2)}")
    
    sx = 1 if x1 < x2 else -1 # step in x direction
    sy = 1 if y1 < y2 else -1 # step in y direction

    yield(x1, y1)
    
    x, y = x1, y1
    while x != x2:
        x += sx
        yield (x, y)

    x, y = x1, y1
    while y != y2:
        y += sy
        yield (x, y)

    x, y = x1, y2
    while x != x2:
        x += sx
        yield (x, y)

    x, y = x2, y1
    while y != y2:
        y += sy
        yield (x, y)

points = []
with input_file.open(mode="r", encoding="utf-8") as file:
    for line in file:
        points.append(list(map(int, reversed(line.strip().split(",")) )))

rectangle_areas = calcualte_rectangle_areas_sorted(points)
print(f"possible rectanges: {len(rectangle_areas)}")
print(f"Part1 answer: {rectangle_areas[0][0]}")
print(f"Part2 answer: {find_greatest_area_avoiding_exclusions2(rectangle_areas, generate_inclusions(points))}")


possible rectanges: 122760
Part1 answer: 4750176210
Part2 answer: 4558544742


In [35]:
from fastquadtree import RectQuadTree

rqt = RectQuadTree(bounds=(1, 2, 7, 11), capacity=20, dtype="i32")

rqt.insert((5, 2, 3, 2))
rqt.insert((1, 7, 3, 7))
rqt.insert((7, 9, 5, 9))
rqt.insert((1, 11, 7, 11))
rqt.insert((3, 3, 5, 3))
rqt.insert((3, 4, 5, 4))
rqt.insert((3, 5, 5, 5))
rqt.insert((3, 6, 5, 6))
rqt.insert((5, 7, 2, 7))
rqt.insert((5, 2, 3, 2))
rqt.insert((1, 7, 3, 7))
rqt.insert((7, 9, 5, 9))
rqt.insert((1, 11, 7, 11))
rqt.insert((3, 3, 5, 3))
rqt.insert((3, 4, 5, 4))
rqt.insert((3, 5, 5, 5))
rqt.insert((3, 6, 5, 6))
rqt.insert((5, 7, 2, 7))
rqt.insert((5, 2, 3, 2))
rqt.insert((1, 7, 3, 7))
rqt.insert((7, 9, 5, 9))
rqt.insert((1, 11, 7, 11))
rqt.insert((3, 3, 5, 3))
rqt.insert((3, 4, 5, 4))
rqt.insert((3, 5, 5, 5))
rqt.insert((3, 6, 5, 6))
rqt.insert((2, 7, 5, 7))

print("Count:", rqt.count_items())
print("Range hits:", rqt.query((10, 13, 10, 13)))
print("Range hits:", rqt.query((5, 7, 3, 7)))

Count: 22
Range hits: []
Range hits: [(26, 2, 7, 5, 7)]


In [41]:
len(list(enumerate_rectange_border_coordinates(85979, 83297, 17783, 13680)))

275627

In [21]:
from intervaltree import Interval, IntervalTree

x=5
y=8

#(5,7) (3,7)
yit = IntervalTree()
yit[7:8] = True

xit = IntervalTree()
xit[5:7] = yit


#it.merge_overlaps()
print(xit[x])

{Interval(5, 7, IntervalTree([Interval(7, 8, True)]))}
